In [ ]:
### Training 

import cv2
import pytesseract
from ultralytics import YOLO
import os

model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

model.train(model.train(
    data='/home/pegasus/Documents/NUMBERplates/My First Project.v12i.yolov8W/data.yaml',  # updated data.yaml
    epochs=50,            # adjust as needed
    imgsz=640,            # higher resolution for more details
    batch=16,             # reduce if low GPU memory
    patience=5,           # stop early if no improvement
    save=True,            # save checkpoints
    name='retrained_model', # experiment name
    resume=False          # don't resume previous run
))



In [ ]:


# Load trained YOLO model
model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

# Directory containing test images
image_folder = "/home/pegasus/Documents/NUMBERplates/testimgs"
output_txt = "ocr_results_simple.txt"

# Create output file
with open(output_txt, "w") as f:
    for image_file in os.listdir(image_folder):
        if not image_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue

        image_path = os.path.join(image_folder, image_file)

        # Run YOLO prediction
        results = model.predict(image_path, imgsz=640, conf=0.3)

        # Load original image
        img = cv2.imread(image_path)

        for r in results:
            for i, box in enumerate(r.boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                plate_img = img[y1:y2, x1:x2]

                # Resize and basic grayscale conversion
                plate_img = cv2.resize(plate_img, (400, 100))
                gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)

                # Basic threshold to help Tesseract
                _, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

                # Run Tesseract OCR
                config = "--psm 7"
                text = pytesseract.image_to_string(thresh, config=config).strip()

                # Log results
                f.write(f"{image_file} - Plate {i+1}: {text}\n")
                print(f"{image_file} - Plate {i+1}: {text}")


In [ ]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image, ImageDraw

# Load trained YOLO model
model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

# Folder with input images
image_folder = "/home/pegasus/Documents/NUMBERplates/testimgs"

# Loop through all images
for image_file in os.listdir(image_folder):
    if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    image_path = os.path.join(image_folder, image_file)

    # Run YOLO prediction
    results = model.predict(image_path, imgsz=640, conf=0.3)

    # Load image with OpenCV, convert to RGB
    cv_img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

    # Convert to PIL image for drawing
    pil_img = Image.fromarray(rgb_img)
    draw = ImageDraw.Draw(pil_img)

    # Draw all detected bounding boxes
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            draw.rectangle([(x1, y1), (x2, y2)], outline="red", width=3)

    # Show image with bounding boxes
    pil_img.show()


In [ ]:
## saving cropped images
import os
import cv2
from ultralytics import YOLO
from PIL import Image, ImageDraw

# Load trained YOLO model
model = YOLO('/home/pegasus/Documents/NUMBERplates/best2.pt')

# Input folder with test images
image_folder = "/home/pegasus/Documents/NUMBERplates/testimgs"

# Output folder for cropped number plates
output_folder = "/home/pegasus/Documents/NUMBERplates/cropped_plates"
os.makedirs(output_folder, exist_ok=True)

# Iterate over images
for image_file in os.listdir(image_folder):
    if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    image_path = os.path.join(image_folder, image_file)
    results = model.predict(image_path, imgsz=640, conf=0.3)

    # Load image using OpenCV
    img = cv2.imread(image_path)

    for r_index, r in enumerate(results):
        for i, box in enumerate(r.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cropped = img[y1:y2, x1:x2]

            # Save cropped plate image
            cropped_filename = f"{os.path.splitext(image_file)[0]}_plate{i+1}.jpg"
            cropped_path = os.path.join(output_folder, cropped_filename)
            cv2.imwrite(cropped_path, cropped)

print(f"✅ Cropped plates saved to: {output_folder}")


## OCR


In [ ]:
import os
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Replace with your folder path
folder_path = "/home/pegasus/Documents/NUMBERplates/cropped_plates"

# Tesseract config - PSM 6 is good for text blocks, 7 for single line
tess_config = "--oem 3 --psm 6"

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (400, 100))  # Resize to a fixed size

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Bilateral Filter to reduce noise but preserve edges
    blur = cv2.bilateralFilter(gray, 11, 17, 17)

    # Histogram Equalization
    eq = cv2.equalizeHist(blur)

    # Adaptive Thresholding (black text on white bg)
    thresh = cv2.adaptiveThreshold(eq, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 11, 2)

    # Morphological Opening to remove small noise
    kernel = np.ones((2, 2), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    return opening

def ocr_on_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            full_path = os.path.join(folder_path, filename)
            processed_img = preprocess_image(full_path)

            # OCR
            text = pytesseract.image_to_string(processed_img, config=tess_config).strip()

            print(f"\n📄 {filename}: {text}")
            # To debug, show image
            # Image.fromarray(processed_img).show()

# Run the function
ocr_on_folder(folder_path)


In [ ]:
import cv2
import pytesseract
import easyocr
from paddleocr import PaddleOCR
import matplotlib.pyplot as plt
import numpy as np

# --- Load image ---
img_path = '/home/pegasus/Documents/NUMBERplates/cropped_plates/4064_plate1.jpg'  # <-- change this
image = cv2.imread(img_path)

# --- Preprocessing ---
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5,5), 0)
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                               cv2.THRESH_BINARY_INV, 11, 2)

# Display the processed image
plt.imshow(thresh, cmap='gray')
plt.title("Preprocessed")
plt.axis('off')
plt.show()

# --- OCR 1: Tesseract ---
tess_text = pytesseract.image_to_string(thresh, config='--psm 6')
print("🔹 Tesseract Output:")
print(tess_text.strip())

# --- OCR 2: EasyOCR ---
print("\n🔹 EasyOCR Output:")
reader = easyocr.Reader(['en'])
results = reader.readtext(image)
easy_text = ' '.join([res[1] for res in results])
print(easy_text.strip())

# --- OCR 3: PaddleOCR ---
print("\n🔹 PaddleOCR Output:")
ocr = PaddleOCR(use_angle_cls=True, lang='en')
paddle_results = ocr.ocr(img_path, cls=True)
paddle_text = ' '.join([line[1][0] for line in paddle_results[0]])
print(paddle_text.strip())


## PADDLE OCR


In [ ]:
"""from paddleocr import PaddleOCR, draw_ocr
import os
import cv2
from matplotlib import pyplot as plt

# ✅ Set font path manually (Linux/Mac/Windows users adjust accordingly)
font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"  # Linux
# font_path = "C:/Windows/Fonts/arial.ttf"  # Uncomment this line on Windows

# Initialize OCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Folder with images
image_folder = '/home/pegasus/Documents/NUMBERplates/cropped_plates'  # ⬅️ Replace with your actual folder path
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)

    result = ocr.ocr(image_path, cls=True)

    print(f"\n--- {image_name} ---")
    if result and result[0]:
        for word_info in result[0]:
            print(word_info[1][0])

        # Draw OCR output
        boxes = [line[0] for line in result[0]]
        txts = [line[1][0] for line in result[0]]
        scores = [line[1][1] for line in result[0]]

        image_with_boxes = draw_ocr(image, boxes, txts, scores, font_path=font_path)
        image_with_boxes = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(10, 8))
        plt.imshow(image_with_boxes)
        plt.title(f"OCR Result - {image_name}")
        plt.axis('off')
        plt.show()
    else:
        print("No text detected.")"""


In [ ]:
from paddleocr import PaddleOCR
import os

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Path to your folder of plate images
image_folder = '/home/pegasus/Documents/NUMBERplates/cropped_plates'
output_txt_path = '/home/pegasus/Documents/NUMBERplates/ocr_results.txt'

# Get image list
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Open the output file
with open(output_txt_path, 'w', encoding='utf-8') as file:
    # Iterate through each image and extract text
    for image_name in image_files:
        image_path = os.path.join(image_folder, image_name)
        result = ocr.ocr(image_path, cls=True)

        plate_text = []
        if result and result[0]:
            for line in result[0]:
                plate_text.append(line[1][0])

        # Join detected text into a single string
        final_text = ' '.join(plate_text).strip()
        final_output = final_text if final_text else 'No text detected'

        # Print to console and write to file
        print(f"{image_name} -> {final_output}")
        file.write(f"{image_name} -> {final_output}\n")

print(f"\n✅ All OCR results saved to: {output_txt_path}")


## full script

In [1]:
import os
import cv2
from ultralytics import YOLO
from paddleocr import PaddleOCR

# ----------- Configuration -----------
YOLO_MODEL_PATH = '/home/pegasus/Documents/NUMBERplates/best2.pt'
INPUT_IMAGE_FOLDER = '/home/pegasus/Documents/NUMBERplates/testimgs'
CROPPED_OUTPUT_FOLDER = '/home/pegasus/Documents/NUMBERplates/cropped_plates'
SUMMARY_OUTPUT_FILE = '/home/pegasus/Documents/NUMBERplates/detected_plates.txt'
ANNOTATED_OUTPUT_FOLDER = '/home/pegasus/Documents/NUMBERplates/output_images'
CONFIDENCE_THRESHOLD = 0.3
SUPPORTED_EXTENSIONS = ('.jpg', '.jpeg', '.png')

# ----------- Setup -----------
os.makedirs(CROPPED_OUTPUT_FOLDER, exist_ok=True)
os.makedirs(ANNOTATED_OUTPUT_FOLDER, exist_ok=True)

# Load models
print("🔍 Loading YOLO model...")
model = YOLO(YOLO_MODEL_PATH)

print("🔠 Initializing PaddleOCR...")
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Get list of test images
image_files = [f for f in os.listdir(INPUT_IMAGE_FOLDER) if f.lower().endswith(SUPPORTED_EXTENSIONS)]
print(f"🖼️ Found {len(image_files)} images.")

# Open summary file for writing
with open(SUMMARY_OUTPUT_FILE, 'w', encoding='utf-8') as summary_file:

    for image_file in image_files:
        image_path = os.path.join(INPUT_IMAGE_FOLDER, image_file)
        img = cv2.imread(image_path)

        if img is None:
            print(f"⚠️ Skipping unreadable image: {image_file}")
            summary_file.write(f"{image_file} -> Error reading image\n")
            continue

        results = model.predict(image_path, imgsz=640, conf=CONFIDENCE_THRESHOLD)
        detected_texts = []

        for r_index, result in enumerate(results):
            for i, box in enumerate(result.boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                cropped = img[y1:y2, x1:x2]

                # Save cropped plate
                cropped_filename = f"{os.path.splitext(image_file)[0]}_plate{i+1}.jpg"
                cropped_path = os.path.join(CROPPED_OUTPUT_FOLDER, cropped_filename)
                cv2.imwrite(cropped_path, cropped)

                # OCR
                ocr_result = ocr.ocr(cropped_path, cls=True)
                plate_text = []

                if ocr_result and ocr_result[0]:
                    for line in ocr_result[0]:
                        plate_text.append(line[1][0])

                final_text = ' '.join(plate_text).strip()
                detected_texts.append(final_text if final_text else "No text detected")

                # Draw rectangle and label on original image
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = final_text if final_text else "No text"
                cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (0, 255, 0), 2, cv2.LINE_AA)

        # Save annotated image
        output_image_path = os.path.join(ANNOTATED_OUTPUT_FOLDER, image_file)
        cv2.imwrite(output_image_path, img)

        # Save result line to text file
        summary_line = f"{image_file} -> {' | '.join(detected_texts) if detected_texts else 'No plate detected'}"
        print(summary_line)
        summary_file.write(summary_line + "\n")

print(f"\n✅ Annotated images saved to: {ANNOTATED_OUTPUT_FOLDER}")
print(f"✅ Detection summary saved to: {SUMMARY_OUTPUT_FILE}")


/home/pegasus/.local/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
/home/pegasus/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 Loading YOLO model...
🔠 Initializing PaddleOCR...
[2025/05/18 13:32:41] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/pegasus/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/pegasus/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inv